# Deep Learning group assignment
Group name: Angry Birds

Group members:
- Nienke Reijnen: 2117034
- Andrea Ciavatti: 2115635
- Niels Boonstra: 1451294
- Yannick Lankhorst: 2052754
- Thom Zoomer:2059225
- Anne Barnasconi: 2053988

## Setting up the environment

Before running, make sure to also have installed the following packages (according to lab 8 instructions):
- pip install imageio
- pip install future
- pip install tensorboard

In [18]:
import os
import shutil
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset
import json
from PIL import Image

In [2]:
device, device_name = (torch.device("cuda"), torch.cuda.get_device_name(0)) if torch.cuda.is_available() else (torch.device("cpu"), "CPU")
print(f"Device: {device}, {device_name}")

Device: cpu, CPU


## Data loading & preprocessing

- Correct implementation of data loaders for images and annotations for your specific object detection model
- Use of data augmentation techniques
- Appropriate shuffling  and batching of data
- Conduct an online search for relevant open-source datasets, and if you can find them, use them in your application as additional training data (to improve generalization)

### Data loading

In [4]:
################################
### Defining transformations ###
################################

transform = None # here we define transforms we want to apply when loading the data


In [19]:
######################################
### Defining a CustomDataset class ###
######################################


class CustomDataset(Dataset):
    def __init__(self, data_path, transform):
        """
        Initialize the custom dataset.
        Works for both the train data and the test data.
        """
        self.images_dir = os.path.join(data_path, "images")
        self.transform = transform
        annotations_file = data_path + "/annotations.json"
        with open(annotations_file, 'r') as f:
            annotations_list = json.load(f)
       
        # We need to extract the bounding boxes of the annotations from the JSON file and store them as [x_min, y_min, x_max, y_max] tensors
        self.data = []
        for entry in annotations_list:
            image_name = entry['OriginalFileName']
            annotation_data = entry['AnnotationData']
            bird_boxes = self.extract_bird_boxes(annotation_data)
            self.data.append({'imagename': image_name, 'bird_boxes_tensor': bird_boxes})

        # Note: we should not load all the images into a tensor here, as it would take too much memory. We load images into a tensor in the __getitem__ method.


    def extract_bird_boxes(self, annotation_data):
        """
        Extract the coordinates of the birds from the annotation data in the JSON file and return it as a tensor.
        """
        bird_boxes = []
        for entry in annotation_data:
            if entry['Label'] == 'Bird':
                coordinates_list = entry['Coordinates']
                x_coordinates = [point['X'] for point in coordinates_list]
                y_coordinates = [point['Y'] for point in coordinates_list]
                x_min, x_max = min(x_coordinates), max(x_coordinates)
                y_min, y_max = min(y_coordinates), max(y_coordinates)
                bird_boxes.append([x_min, y_min, x_max, y_max])

        return torch.tensor(bird_boxes, dtype=torch.float32) # Shape: (num_birds, 4)


    def __len__(self):
        """
        Return the size of the dataset, i.e. the number of images.
        """
        return len(self.data)


    def __getitem__(self, index):
        """
        Load an image and its corresponding annotations.
        """
        item = self.data[index]
        image_path = os.path.join(self.images_dir, item['imagename'])
        image = Image.open(image_path).convert("RGB")

        # If we are using data augmentations, we should apply them here:
        if self.transform:
            image = self.transform(image)
        
        return image, item['bird_boxes_tensor']


In [20]:
batch_size = 32

train_data_path = "scarecrow_dataset/train"
test_data_path = "scarecrow_dataset/test"

train_data = CustomDataset("scarecrow_dataset/train", transform)
test_data = CustomDataset("scarecrow_dataset/test", transform)
train_data, valid_data = torch.utils.data.random_split(train_data, [0.8, 0.2])

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(valid_data, batch_size=batch_size)
test_loader  = DataLoader(test_data, batch_size=batch_size)